## Installations and Imports

In [2]:
import networkx as nx
from tqdm.auto import tqdm
from time import sleep
import random
import pickle
import warnings
warnings.filterwarnings("ignore")
from networkx.algorithms.approximation import clique
import pandas as pd

In [ ]:
!touch "cookies.txt"

In [ ]:
!pip install git+https://github.com/kevinzg/facebook-scraper.git --quiet

In [ ]:
from facebook_scraper import get_friends

# JS Code for Group Members Scraping

In [ ]:
var lastScrollHeight = 0; 
function autoScroll() { 
  var sh = document.documentElement.scrollHeight; 
  if (sh != lastScrollHeight) { 
    lastScrollHeight = sh; 
    document.documentElement.scrollTop = sh; 
  } 
} 
window.setInterval(autoScroll, 100);

function exportToCsv(e,t){for(var n="",o=0;o<t.length;o++)n+=function(e){for(var t="",n=0;n<e.length;n++){var o=null===e[n]||void 0===e[n]?"":e[n].toString(),o=(o=e[n]instanceof Date?e[n].toLocaleString():o).replace(/"/g,'""');0<n&&(t+=","),t+=o=0<=o.search(/("|,|\n)/g)?'"'+o+'"':o}return t+"\n"}(t[o]);var r=new Blob([n],{type:"text/csv;charset=utf-8;"}),i=document.createElement("a");void 0!==i.download&&(r=URL.createObjectURL(r),i.setAttribute("href",r),i.setAttribute("download",e),document.body.appendChild(i),i.click(),document.body.removeChild(i))}function buildCTABtn(){var e=document.createElement("div"),t=(e.setAttribute("style",["position: fixed;","top: 0;","left: 0;","z-index: 10;","width: 100%;","height: 100%;","pointer-events: none;"].join("")),document.createElement("div")),n=(t.setAttribute("style",["position: absolute;","bottom: 30px;","right: 130px;","color: white;","min-width: 150px;","background: var(--primary-button-background);","border-radius: var(--button-corner-radius);","padding: 0px 12px;","cursor: pointer;","font-weight:600;","font-size:15px;","display: inline-flex;","pointer-events: auto;","height: 36px;","align-items: center;","justify-content: center;"].join("")),document.createTextNode("Download ")),o=document.createElement("span"),r=(o.setAttribute("id","fb-group-scraper-number-tracker"),o.textContent="0",document.createTextNode(" members"));return t.appendChild(n),t.appendChild(o),t.appendChild(r),t.addEventListener("click",function(){var e=(new Date).toISOString();exportToCsv("groupMemberExport-".concat(e,".csv"),window.members_list)}),e.appendChild(t),document.body.appendChild(e),e}function processResponse(e){var t,n;if(null!==(t=null==e?void 0:e.data)&&void 0!==t&&t.group)o=e.data.group;else{if("Group"!==(null===(t=null===(t=null==e?void 0:e.data)||void 0===t?void 0:t.node)||void 0===t?void 0:t.__typename))return;o=e.data.node}if(null!==(t=null==o?void 0:o.new_members)&&void 0!==t&&t.edges)n=o.new_members.edges;else if(null!==(e=null==o?void 0:o.new_forum_members)&&void 0!==e&&e.edges)n=o.new_forum_members.edges;else{if(null===(t=null==o?void 0:o.search_results)||void 0===t||!t.edges)return;n=o.search_results.edges}var e=n.map(function(e){var t=e.node,n=t.id,o=t.name,r=t.bio_text,i=t.url,s=t.profile_picture,t=t.__isProfile,d=(null===(d=null==e?void 0:e.join_status_text)||void 0===d?void 0:d.text)||(null===(d=null===(d=null==e?void 0:e.membership)||void 0===d?void 0:d.join_status_text)||void 0===d?void 0:d.text),e=null===(e=e.node.group_membership)||void 0===e?void 0:e.associated_group.id;return[n,o,i,(null==r?void 0:r.text)||"",(null==s?void 0:s.uri)||"",e,d||"",t]}),o=((t=window.members_list).push.apply(t,e),document.getElementById("fb-group-scraper-number-tracker"));o&&(o.textContent=window.members_list.length.toString())}function parseResponse(e){var n=[];try{n.push(JSON.parse(e, function(k, v){return (typeof v === "string") ? String(v) : v; 
}))}catch(t){var o=e.split("\n");if(o.length<=1)return void console.error("Fail to parse API response",t);for(var r=0;r<o.length;r++){var i=o[r];try{n.push(JSON.parse(i))}catch(e){console.error("Fail to parse API response",t)}}}for(var t=0;t<n.length;t++)processResponse(n[t])}function main(){buildCTABtn();var e=XMLHttpRequest.prototype.send;XMLHttpRequest.prototype.send=function(){this.addEventListener("readystatechange",function(){this.responseURL.includes("/api/graphql/")&&4===this.readyState&&parseResponse(this.responseText)},!1),e.apply(this,arguments)}}window.members_list=window.members_list||[["Profile Id","Full Name","ProfileLink","Bio","Image Src","Groupe Id","Group Joining Text","Profile Type"]],main();

# Graph Extraction

In [ ]:
# read a csv named "egy_geeks_sample.csv"
df = pd.read_csv("whole_data_5.csv")

num_fr = 100

profiles_n_friends = {}

count, err = 0, 0
pbar = tqdm(range(len(df)), total=len(df), leave=True)


for i in pbar:
  try:
    profile = str(df.iloc[i]["Profile Id"])
    profile= profile.replace("A", "")
    pbar.set_postfix_str(s=f"count={count}, errors={err}")
    pbar.refresh()
    sleep(0.01)

    
    fr = [f for f in get_friends(profile, cookies="cookies.txt", friends=num_fr)]

    # slow down persistent requests for friends
    if random.random() > 0.5:
        sleep(0.1)
    
    if fr:
        profiles_n_friends[profile] = []
        for f in fr:
            profiles_n_friends[profile].append(f['id'])
        count += 1
    else:
        err += 1
  except:
    err += 1
    if profile in profiles_n_friends:
      profiles_n_friends.pop(profile)
    continue
    

# Cleaning Dictionary of users and their friends

In [ ]:
new_pr_n_fr = {}
for k, j in person.items():
    if k not in new_pr_n_fr:
        new_pr_n_fr[int(k)] = []
    for j_ in j:
        if j_ != '':
            new_pr_n_fr[int(k)].append(j_)

for k, j in profiles_n_friends.items():
    if k not in new_pr_n_fr:
        new_pr_n_fr[int(k)] = []
    for j_ in j:
        if j_ != '' and j_ not in j:
            new_pr_n_fr[int(k)].append(j_)

Save Dictionary as pickle file

In [30]:
# save dictionary to person_data.pkl file
with open('whole_data_5.pkl', 'wb') as fp:
    pickle.dump(profiles_n_friends, fp)
    print('dictionary saved successfully to file')

dictionary saved successfully to file


# Merge 6 dictionaries

In [11]:
merged_dict = {}
merged_dict.update(Group1)
merged_dict.update(Group2)
merged_dict.update(Group3)
merged_dict.update(Group4)
merged_dict.update(Group5)
merged_dict.update(Group6)
len(merged_dict)

9313

In [10]:
new_pr_n_fr = {}
for k, j in merged_dict.items():
    if k not in new_pr_n_fr:
        new_pr_n_fr[int(k)] = []
    for j_ in j:
        if j_ != '':
            new_pr_n_fr[int(k)].append(j_)

In [11]:
len(new_pr_n_fr)

9313

In [13]:
for em in e_list:
    new_pr_n_fr.pop(em)